In [95]:
import streamlit as st
from PIL import Image
import os, sys
import pandas as pd
import json
import pymysql
from sqlalchemy import create_engine
import pymysql
from getpass import getpass
from mysql_driver import MySQL
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


In [42]:
dir = os.path.dirname
src_path = dir(os.getcwd())
sep = os.sep
totem = dir(src_path)

In [43]:
configbd_json_path = totem + sep + 'config' + sep + 'bd_info.json'
with open(configbd_json_path, "r") as configbd_json_readed:
    json_configbd = json.load(configbd_json_readed)
IP_DNS = json_configbd["IP_DNS"]
USER = json_configbd["USER"]
PASSWORD = json_configbd["PASSWORD"]
BD_NAME = json_configbd["BD_NAME"]
PORT = json_configbd["PORT"]

In [44]:
mysql_db = MySQL(IP_DNS=IP_DNS, USER=USER, PASSWORD=PASSWORD, BD_NAME=BD_NAME, PORT=PORT)
conn = mysql_db.connect()

Connected to MySQL server [australia_fires]


In [45]:
select_sql = """SELECT * FROM australia_fires.fire_archive_M6_96619"""
select_result = mysql_db.execute_get_sql(sql=select_sql)

Executing:
 SELECT * FROM australia_fires.fire_archive_M6_96619


In [66]:
df = pd.read_sql('SELECT * FROM australia_fires.fire_archive_M6_96619', con=conn)

In [67]:
df

,fire_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,fire_type
0,0,-11.807,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,MODIS,48,6.3,297.3,6.6,D,0
1,1,-11.7924,142.0850,319.3,1.0,1.0,2019-08-01,56,Terra,MODIS,71,6.3,297.3,11.3,D,0
2,2,-12.8398,132.8744,311.6,3.1,1.7,2019-08-01,57,Terra,MODIS,42,6.3,298.7,23.1,D,0
3,3,-14.4306,143.3035,310.1,1.1,1.1,2019-08-01,57,Terra,MODIS,33,6.3,296.1,6.5,D,0
4,4,-12.4953,131.4897,310.3,4.0,1.9,2019-08-01,57,Terra,MODIS,36,6.3,298.8,27.6,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36006,36006,-28.0656,131.6334,312.7,1.2,1.1,2019-09-30,1644,Aqua,MODIS,85,6.3,292.9,14.2,N,0
36007,36007,-28.2625,128.5494,304.9,1.0,1.0,2019-09-30,1644,Aqua,MODIS,27,6.3,290.9,6.4,N,0
36008,36008,-28.2941,128.5123,301.7,1.0,1.0,2019-09-30,1644,Aqua,MODIS,43,6.3,290.4,5.5,N,0
36009,36009,-28.3001,128.5530,303.6,1.0,1.0,2019-09-30,1644,Aqua,MODIS,41,6.3,290.3,5.7,N,0


In [68]:
df.corr()

,fire_id,longitude,brightness,scan,track,acq_time,confidence,version,bright_t31,frp,fire_type
fire_id,1.000000,0.103756,0.158104,-0.004500,-0.005544,0.054372,0.075489,NaN,0.097194,0.094743,-0.004516
longitude,0.103756,1.000000,-0.104097,-0.051840,-0.054767,0.116368,-0.035529,NaN,-0.380406,0.000449,0.005580
brightness,0.158104,-0.104097,1.000000,-0.105940,-0.108572,-0.323271,0.614691,NaN,0.606844,0.661131,-0.054160
scan,-0.004500,-0.051840,-0.105940,1.000000,0.982484,-0.026877,-0.103808,NaN,-0.094356,0.292117,-0.024130
track,-0.005544,-0.054767,-0.108572,0.982484,1.000000,-0.024399,-0.105949,NaN,-0.092064,0.286563,-0.024460
acq_time,0.054372,0.116368,-0.323271,-0.026877,-0.024399,1.000000,-0.070012,NaN,-0.552448,-0.073658,0.007624
confidence,0.075489,-0.035529,0.614691,-0.103808,-0.105949,-0.070012,1.000000,NaN,0.238506,0.355388,-0.041632
version,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bright_t31,0.097194,-0.380406,0.606844,-0.094356,-0.092064,-0.552448,0.238506,NaN,1.000000,0.216824,-0.054393
frp,0.094743,0.000449,0.661131,0.292117,0.286563,-0.073658,0.355388,NaN,0.216824,1.000000,-0.026688


In [69]:
df.drop(['version', 'instrument', 'fire_id'], axis=1, inplace=True)

In [70]:
df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,bright_t31,frp,daynight,fire_type
0,-11.807,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,48,297.3,6.6,D,0
1,-11.7924,142.0850,319.3,1.0,1.0,2019-08-01,56,Terra,71,297.3,11.3,D,0
2,-12.8398,132.8744,311.6,3.1,1.7,2019-08-01,57,Terra,42,298.7,23.1,D,0
3,-14.4306,143.3035,310.1,1.1,1.1,2019-08-01,57,Terra,33,296.1,6.5,D,0
4,-12.4953,131.4897,310.3,4.0,1.9,2019-08-01,57,Terra,36,298.8,27.6,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36006,-28.0656,131.6334,312.7,1.2,1.1,2019-09-30,1644,Aqua,85,292.9,14.2,N,0
36007,-28.2625,128.5494,304.9,1.0,1.0,2019-09-30,1644,Aqua,27,290.9,6.4,N,0
36008,-28.2941,128.5123,301.7,1.0,1.0,2019-09-30,1644,Aqua,43,290.4,5.5,N,0
36009,-28.3001,128.5530,303.6,1.0,1.0,2019-09-30,1644,Aqua,41,290.3,5.7,N,0


,fire_id,longitude,brightness,scan,track,acq_time,confidence,bright_t31,frp,fire_type
fire_id,1.000000,0.103756,0.158104,-0.004500,-0.005544,0.054372,0.075489,0.097194,0.094743,-0.004516
longitude,0.103756,1.000000,-0.104097,-0.051840,-0.054767,0.116368,-0.035529,-0.380406,0.000449,0.005580
brightness,0.158104,-0.104097,1.000000,-0.105940,-0.108572,-0.323271,0.614691,0.606844,0.661131,-0.054160
scan,-0.004500,-0.051840,-0.105940,1.000000,0.982484,-0.026877,-0.103808,-0.094356,0.292117,-0.024130
track,-0.005544,-0.054767,-0.108572,0.982484,1.000000,-0.024399,-0.105949,-0.092064,0.286563,-0.024460
acq_time,0.054372,0.116368,-0.323271,-0.026877,-0.024399,1.000000,-0.070012,-0.552448,-0.073658,0.007624
confidence,0.075489,-0.035529,0.614691,-0.103808,-0.105949,-0.070012,1.000000,0.238506,0.355388,-0.041632
bright_t31,0.097194,-0.380406,0.606844,-0.094356,-0.092064,-0.552448,0.238506,1.000000,0.216824,-0.054393
frp,0.094743,0.000449,0.661131,0.292117,0.286563,-0.073658,0.355388,0.216824,1.000000,-0.026688
fire_type,-0.004516,0.005580,-0.054160,-0.024130,-0.024460,0.007624,-0.041632,-0.054393,-0.026688,1.000000


In [71]:
le = LabelEncoder()
df["satellite"] = le.fit_transform(df["satellite"])
df["daynight"] = le.fit_transform(df["daynight"])

In [72]:
df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,bright_t31,frp,daynight,fire_type
0,-11.807,142.0583,313.0,1.0,1.0,2019-08-01,56,1,48,297.3,6.6,0,0
1,-11.7924,142.0850,319.3,1.0,1.0,2019-08-01,56,1,71,297.3,11.3,0,0
2,-12.8398,132.8744,311.6,3.1,1.7,2019-08-01,57,1,42,298.7,23.1,0,0
3,-14.4306,143.3035,310.1,1.1,1.1,2019-08-01,57,1,33,296.1,6.5,0,0
4,-12.4953,131.4897,310.3,4.0,1.9,2019-08-01,57,1,36,298.8,27.6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36006,-28.0656,131.6334,312.7,1.2,1.1,2019-09-30,1644,0,85,292.9,14.2,1,0
36007,-28.2625,128.5494,304.9,1.0,1.0,2019-09-30,1644,0,27,290.9,6.4,1,0
36008,-28.2941,128.5123,301.7,1.0,1.0,2019-09-30,1644,0,43,290.4,5.5,1,0
36009,-28.3001,128.5530,303.6,1.0,1.0,2019-09-30,1644,0,41,290.3,5.7,1,0


In [73]:
features = ['latitude', 'longitude', 'brightness', 'scan', 'track', 'acq_time', 'confidence', 'bright_t31', 'frp']
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
y = df.loc[:,['fire_type']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [78]:
df[features] = x

In [79]:
df

,brightness,scan,track,acq_time,satellite,confidence,bright_t31,frp,daynight,fire_type,latitude,longitude
0,-0.829309,-0.763014,-0.868600,-0.992093,1,-0.843473,-0.415364,-0.482583,0,0,1.003893,0.337627
1,-0.497600,-0.763014,-0.868600,-0.992093,1,0.148778,-0.415364,-0.431650,0,0,1.005902,0.340510
2,-0.903023,1.817623,1.955703,-0.990270,1,-1.102321,-0.271395,-0.303777,0,0,0.861745,-0.654019
3,-0.982001,-0.640127,-0.465128,-0.990270,1,-1.490593,-0.538766,-0.483666,0,0,0.642798,0.472079
4,-0.971470,2.923610,2.762646,-0.990270,1,-1.361169,-0.261112,-0.255012,0,0,0.909160,-0.803534
...,...,...,...,...,...,...,...,...,...,...,...,...
36006,-0.845105,-0.517239,-0.465128,1.903807,0,0.752757,-0.867838,-0.400224,1,0,-1.233834,-0.788018
36007,-1.255793,-0.763014,-0.868600,1.903807,0,-1.749441,-1.073508,-0.484750,1,0,-1.260934,-1.121017
36008,-1.424280,-0.763014,-0.868600,1.903807,0,-1.059180,-1.124925,-0.494503,1,0,-1.265283,-1.125023
36009,-1.324240,-0.763014,-0.868600,1.903807,0,-1.145462,-1.135208,-0.492336,1,0,-1.266109,-1.120629


In [75]:
df.drop(['acq_date'], axis=1, inplace=True)

In [81]:
df.drop(['latitude', 'longitude'], axis=1, inplace=True)

In [82]:
df

,brightness,scan,track,acq_time,satellite,confidence,bright_t31,frp,daynight,fire_type
0,-0.829309,-0.763014,-0.868600,-0.992093,1,-0.843473,-0.415364,-0.482583,0,0
1,-0.497600,-0.763014,-0.868600,-0.992093,1,0.148778,-0.415364,-0.431650,0,0
2,-0.903023,1.817623,1.955703,-0.990270,1,-1.102321,-0.271395,-0.303777,0,0
3,-0.982001,-0.640127,-0.465128,-0.990270,1,-1.490593,-0.538766,-0.483666,0,0
4,-0.971470,2.923610,2.762646,-0.990270,1,-1.361169,-0.261112,-0.255012,0,0
...,...,...,...,...,...,...,...,...,...,...
36006,-0.845105,-0.517239,-0.465128,1.903807,0,0.752757,-0.867838,-0.400224,1,0
36007,-1.255793,-0.763014,-0.868600,1.903807,0,-1.749441,-1.073508,-0.484750,1,0
36008,-1.424280,-0.763014,-0.868600,1.903807,0,-1.059180,-1.124925,-0.494503,1,0
36009,-1.324240,-0.763014,-0.868600,1.903807,0,-1.145462,-1.135208,-0.492336,1,0


In [83]:
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['Comp1', 'Comp2', 'Comp3', 'Comp4', 'Comp5'])

In [86]:
principalDf

,Comp1,Comp2,Comp3,Comp4,Comp5
0,-0.571288,-1.076144,-1.112505,-0.767874,0.291846
1,-0.038200,-1.160805,-0.627911,-0.424110,-0.378473
2,-0.405661,2.664733,-1.801543,-0.803036,-0.238486
3,-1.075571,-0.666066,-1.126509,-1.135117,0.655882
4,-0.477208,3.998224,-2.062790,-0.906618,-0.307564
...,...,...,...,...,...
36006,-1.485329,-0.712390,0.557610,1.745819,-0.559976
36007,-2.608786,-0.916131,-0.618775,1.101475,1.284417
36008,-2.487667,-0.962784,-0.410505,1.327816,0.725123
36009,-2.470837,-0.959542,-0.406362,1.305930,0.806269


In [93]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [96]:
pipe = Pipeline([('SVC', SVC()),
                  ('reglog', LogisticRegression()),
                  ('randfor', RandomForestClassifier())])

params = { 
    'bootstrap': [True],
    'n_estimators': [200, 300],
    'max_features': ['auto'],
    'max_depth' : [10, 50], 
    'min_samples_leaf': [1, 2], 
    'min_samples_split': [2],
    'random_state': [42],
}


modelo_GS = GridSearchCV(estimator= pipe,
                        param_grid= params,
                        cv = 5,
                        scoring = 'accuracy',
                        verbose=1, 
                        n_jobs=-1)

grids = {'modelo': modelo_GS}

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'SVC()' (type <class 'sklearn.svm._classes.SVC'>) doesn't

In [ ]:
for nombre, grid in grids.items():
    print("#################")
    print("NOMBRE:", nombre)
    print("#################")
    grid.fit(X_train, y_train)